In [1]:
# %pip install psycopg2-binary pandas matplotlib

### Ejecucion de Docker
```bash
docker run -e POSTGRES_USER=edd -e POSTGRES_PASSWORD=PlatoN03 -e POSTGRES_DB=edd_db -p 5432:5432 -d postgres
```

## Conecciónn general 

In [2]:
import psycopg2 as pg
import pandas as pd
from psycopg2.extras import RealDictCursor
import time
import numpy as np

class PostgresConnector:
    def __init__(self):
        self.connection_params = {
            "user": "edd",
            "password": "PlatoN03",
            "host": "localhost",
            "port": "5432",
            "database": "edd_db"
        }
        self.connect()
        
    def connect(self):
        self.conn = pg.connect(**self.connection_params)
        self.cur = self.conn.cursor(cursor_factory=RealDictCursor)
        
    def setup_database(self):
        self.cur.execute("CREATE SCHEMA IF NOT EXISTS db2;")
        
        # Tabla
        create_table_query = """
        CREATE TABLE IF NOT EXISTS db2.spotify_songs (
            track_id VARCHAR PRIMARY KEY,
            track_name VARCHAR,
            track_artist VARCHAR,
            lyrics TEXT,
            search_vector tsvector
        );
        """
        self.cur.execute(create_table_query)
        
        # Índice GIN
        self.cur.execute("""
        CREATE INDEX IF NOT EXISTS idx_songs_search 
        ON db2.spotify_songs USING gin(search_vector);
        """)
        
        # Función de actualización del vector
        self.cur.execute("""
            CREATE OR REPLACE FUNCTION db2.update_search_vector()
            RETURNS trigger AS $$
            BEGIN
                NEW.search_vector = 
                    setweight(to_tsvector('english', COALESCE(NEW.track_id,'')) || 
                            to_tsvector('spanish', COALESCE(NEW.track_id,'')), 'A') ||
                    setweight(to_tsvector('english', COALESCE(NEW.track_name,'')) || 
                            to_tsvector('spanish', COALESCE(NEW.track_name,'')), 'B') ||
                    setweight(to_tsvector('english', COALESCE(NEW.track_artist,'')) || 
                            to_tsvector('spanish', COALESCE(NEW.track_artist,'')), 'C') ||
                    setweight(to_tsvector('english', COALESCE(NEW.lyrics,'')) || 
                            to_tsvector('spanish', COALESCE(NEW.lyrics,'')), 'D');
                RETURN NEW;
            END;
            $$ LANGUAGE plpgsql;
        """)
        
        # Trigger
        self.cur.execute("""
        DROP TRIGGER IF EXISTS trigger_search_vector ON db2.spotify_songs;
        CREATE TRIGGER trigger_search_vector
        BEFORE INSERT OR UPDATE ON db2.spotify_songs
        FOR EACH ROW
        EXECUTE FUNCTION db2.update_search_vector();
        """)
        
        self.conn.commit()

# -----------------------------------------------------------------------------------        
# -----------------------------------------------------------------------------------        
    def populate_tables_from_csv(self, csv_path, sizes=[1000, 2000, 4000, 8000, 16000, 32000, 64000]):
        """Poblar tablas desde CSV duplicando datos si es necesario"""
        
        # Crear schema para pruebas si no existe
        self.cur.execute("CREATE SCHEMA IF NOT EXISTS test;")
        
        print("Leyendo CSV...")
        df_original = pd.read_csv(csv_path)
        total_original = len(df_original)
        print(f"Registros originales: {total_original}")
        
        for size in sizes:
            print(f"\nProcesando tabla para N={size}")
            start_time = time.time()
            table_name = f"test.songs_{size}"
            
            # Crear tabla
            self.cur.execute(f"""
            DROP TABLE IF EXISTS {table_name};
            CREATE TABLE {table_name} (
                track_id VARCHAR PRIMARY KEY,
                track_name VARCHAR,
                track_artist VARCHAR,
                lyrics TEXT,
                search_vector tsvector
            );
            """)
            
            # Calcular repeticiones necesarias
            repetitions = int(np.ceil(size / total_original))
            
            # Crear DataFrame con datos duplicados
            df_list = []
            for i in range(repetitions):
                df_temp = df_original.copy()
                df_temp['track_id'] = df_temp['track_id'] + f'_dup_{i}'
                df_list.append(df_temp)
            
            df_final = pd.concat(df_list, ignore_index=True)
            df_final = df_final.head(size)  # Tomar solo los registros necesarios
            
            # Convertir a lista de tuplas para inserción
            records = df_final[['track_id', 'track_name', 'track_artist', 'lyrics']].values.tolist()
            
            # Insertar por lotes para mejor rendimiento
            batch_size = 1000
            for i in range(0, len(records), batch_size):
                batch = records[i:i + batch_size]
                self.cur.executemany(f"""
                INSERT INTO {table_name} (track_id, track_name, track_artist, lyrics)
                VALUES (%s, %s, %s, %s)
                """, batch)
                self.conn.commit()
            
            # Actualizar vectores de búsqueda
            self.cur.execute(f"""
            UPDATE {table_name}
            SET search_vector = 
                    setweight(to_tsvector('english', COALESCE(track_id,'')) || 
                            to_tsvector('spanish', COALESCE(track_id,'')), 'A') ||
                    setweight(to_tsvector('english', COALESCE(track_name,'')) || 
                            to_tsvector('spanish', COALESCE(track_name,'')), 'B') ||
                    setweight(to_tsvector('english', COALESCE(track_artist,'')) || 
                            to_tsvector('spanish', COALESCE(track_artist,'')), 'C') ||
                    setweight(to_tsvector('english', COALESCE(lyrics,'')) || 
                            to_tsvector('spanish', COALESCE(lyrics,'')), 'D');
            """)
            
            # Crear índice GIN
            self.cur.execute(f"""
            CREATE INDEX idx_songs_search_{size} 
            ON {table_name} USING gin(search_vector);
            """)
            
            self.conn.commit()
            
            # Verificar resultado
            self.cur.execute(f"SELECT COUNT(*) FROM {table_name}")
            actual_size = self.cur.fetchone()['count']
            
            print(f"Tabla {table_name} creada con {actual_size} registros")
            print(f"Tiempo de creación: {time.time() - start_time:.2f} segundos")

# -----------------------------------------------------------------------------------        
# -----------------------------------------------------------------------------------        
    def load_data(self, csv_path):
        self.cur.execute("SELECT COUNT(*) FROM db2.spotify_songs")
        if self.cur.fetchone()['count'] > 0:
            print("Los datos ya están cargados")
            return
            
        # Cargar datos desde CSV
        df = pd.read_csv(csv_path)
        for _, row in df.iterrows():
            self.cur.execute("""
            INSERT INTO db2.spotify_songs (track_id, track_name, track_artist, lyrics)
            VALUES (%s, %s, %s, %s)
            """, (row['track_id'], row['track_name'], row['track_artist'], row['lyrics']))
        
        self.conn.commit()

# -----------------------------------------------------------------------------------        
# -----------------------------------------------------------------------------------                
    def search_songs_with_metrics(self, query: str, test_table: str, k: int = 10):
        try:
            # Limpiar y preparar la consulta
            clean_query = query.lower()
            ts_query = ' | '.join(f"'{word}':*" for word in clean_query.split())
            
            start_time = time.time()
            
            search_query = f"""
            SELECT 
                track_id,
                track_name,
                track_artist,
                lyrics,
                ts_rank(search_vector, to_tsquery('english', %s) || to_tsquery('spanish', %s)) as rank
            FROM {test_table}
            WHERE search_vector @@ (to_tsquery('english', %s) || to_tsquery('spanish', %s))
            ORDER BY rank DESC
            LIMIT %s;
            """
            
            # Pasar todos los parámetros necesarios
            self.cur.execute(search_query, (ts_query, ts_query, ts_query, ts_query, k))
            results = self.cur.fetchall()
            
            end_time = time.time()
            execution_time = (end_time - start_time) * 1000  # Convertir a milisegundos
            
            return {
                'results': results,
                'metrics': {
                    'execution_time_ms': execution_time,
                    'num_results': len(results)
                }
            }
            
        except Exception as e:
            print(f"Error en la búsqueda: {str(e)}")
            return {
                'results': [],
                'metrics': {
                    'execution_time_ms': 0,
                    'num_results': 0,
                    'error': str(e)
                }
            }
# -----------------------------------------------------------------------------------        
# -----------------------------------------------------------------------------------                

    def search(self, query, k=5):
        start_time = time.time()
        
        clean_query = query.lower()
        
        ts_query = ' | '.join(f"'{word}':*" for word in clean_query.split())
        # print(ts_query)
        
        search_query = """
                SELECT 
                    track_id,
                    track_name, 
                    track_artist,
                    lyrics,
                    ctid::text as row_position,
                    ts_rank_cd(search_vector, to_tsquery('english', %s) || to_tsquery('spanish', %s)) as similitud
                FROM db2.spotify_songs
                WHERE search_vector @@ (to_tsquery('english', %s) || to_tsquery('spanish', %s))
                ORDER BY similitud DESC
                LIMIT %s;
                """

        self.cur.execute(search_query, (ts_query, ts_query, ts_query, ts_query, k))
        results = self.cur.fetchall()
        
        return {
            'query_time': time.time() - start_time,
            'results': results
        }

    def __del__(self):
        if hasattr(self, 'cur'):
            self.cur.close()
        if hasattr(self, 'conn'):
            self.conn.close()
  

In [3]:
# Uso

db = PostgresConnector()
# db.setup_database()
# db.load_data("data/spotify_songs.csv")

# Búsqueda
results = db.search("heart just to keep you safe For you, anything for you With you, all the years just fade away Like a dream in my arms", 3)
for result in results['results']:
		print(f"ID de la canción: {result['track_id']}")
		print(f"Nombre de la canción: {result['track_name']}")
		print(f"Artista: {result['track_artist']}")
		# print(f"Letras: {result['lyrics']}")
		print(f"Posición de la fila: {result['row_position']}")
		print(f"Similitud: {result['similitud']}")
		print("---")




ID de la canción: 2xLMifQCjDGFmkHkpNLD9h
Nombre de la canción: SICKO MODE
Artista: Travis Scott
Posición de la fila: (1052,6)
Similitud: 114.6
---
ID de la canción: 5yY9lUy8nbvjM1Uyo1Uqoc
Nombre de la canción: Life Is Good (feat. Drake)
Artista: Future
Posición de la fila: (2135,6)
Similitud: 109.200005
---
ID de la canción: 4aPB8rSxGFISs0o6V2LrTy
Nombre de la canción: Gold Roses (feat. Drake)
Artista: Rick Ross
Posición de la fila: (1507,6)
Similitud: 96.5
---


## Base de datos pruebas

> Creacion de base de datos de pruebas, duplicando los datos existentes en la base de datos de producción. 

Extraer la base de datos de producción, y crear tablas de pruebas con:

N: 10000, 2000, 4000, 8000, 16000, 32000, 64000, 

In [4]:
# db.populate_tables_from_csv("data/spotify_songs.csv")

## Reslutados de tiempos en las tablas de test	

In [5]:
# Ejemplo de uso
results = db.search_songs_with_metrics(query="house", test_table="test.songs_16000", k=10)

if 'error' in results['metrics']:
    print(f"Error: {results['metrics']['error']}")
else:
    print(f"Tiempo de ejecución: {results['metrics']['execution_time_ms']:.2f} ms")
    print(f"Resultados encontrados: {results['metrics']['num_results']}")
    
    # Mostrar algunos resultados
    for r in results['results'][:3]:
        print(f"\nCanción: {r['track_name']}")
        print(f"Artista: {r['track_artist']}")
        print(f"Rank: {r['rank']:.3f}")

Tiempo de ejecución: 277.65 ms
Resultados encontrados: 10

Canción: Our House
Artista: Madness
Rank: 0.403

Canción: House and the Rising Son
Artista: House Of Pain
Rank: 0.331

Canción: Miami 2 Ibiza - Swedish House Mafia vs. Tinie Tempah
Artista: Swedish House Mafia
Rank: 0.329


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time

def run_search_experiments(db_connector):
    """
    Ejecuta experimentos de búsqueda y guarda resultados en CSV
    """
    # Configuración del experimento
    table_sizes = [1000, 2000, 4000, 8000, 16000, 32000, 64000]
    queries = ["love", "in the house tonight"]
    results = []
    
    # Ejecutar búsquedas
    for query in queries:
        for size in table_sizes:
            table_name = f"test.songs_{size}"
            print(f"Ejecutando búsqueda: '{query}' en {table_name}")
            
            # Realizar 3 mediciones para cada combinación
            for iteration in range(3):
                search_result = db_connector.search_songs_with_metrics(
                    query=query,
                    test_table=table_name,
                    k=10
                )
                
                # Guardar cada medición individual
                results.append({
                    'query': query,
                    'table_size': size,
                    'iteration': iteration + 1,
                    'execution_time_ms': search_result['metrics']['execution_time_ms'],
                    'num_results': search_result['metrics']['num_results']
                })
    
    # Convertir a DataFrame y guardar en CSV
    df_results = pd.DataFrame(results)
    
    # Ordenar por query, tamaño de tabla e iteración
    df_results = df_results.sort_values(['query', 'table_size', 'iteration'])
    
    csv_filename = f'search_results_postgres.csv'
    df_results.to_csv(csv_filename, index=False)
    
    print(f"\nResultados guardados en: {csv_filename}")
    return df_results

# Uso:

# db = PostgresConnector()
results_df = run_search_experiments(db)


Ejecutando búsqueda: 'love' en test.songs_1000
Ejecutando búsqueda: 'love' en test.songs_2000
Ejecutando búsqueda: 'love' en test.songs_4000
Ejecutando búsqueda: 'love' en test.songs_8000
Ejecutando búsqueda: 'love' en test.songs_16000
Ejecutando búsqueda: 'love' en test.songs_32000
Ejecutando búsqueda: 'love' en test.songs_64000
Ejecutando búsqueda: 'in the house tonight' en test.songs_1000
Ejecutando búsqueda: 'in the house tonight' en test.songs_2000
Ejecutando búsqueda: 'in the house tonight' en test.songs_4000
Ejecutando búsqueda: 'in the house tonight' en test.songs_8000
Ejecutando búsqueda: 'in the house tonight' en test.songs_16000
Ejecutando búsqueda: 'in the house tonight' en test.songs_32000
Ejecutando búsqueda: 'in the house tonight' en test.songs_64000

Resultados guardados en: search_results_postgres.csv
